        Loading Dataset

In [23]:
import pandas as pd
from datasets import Dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.preprocessing import LabelEncoder

# Load the dataset
df = pd.read_csv('/Users/zainnofal/Desktop/Resume Screening2/Resume/Resume.csv')

# Preview the data
df.head()


,ID,Resume_str,Resume_html,Category
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR
2,33176873,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR
3,27018550,HR SPECIALIST Summary Dedica...,"<div class=""fontsize fontface vmargins hmargin...",HR
4,17812897,HR MANAGER Skill Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",HR


In [24]:
df.drop(columns = ['ID', 'Resume_html'], inplace = True)

In [25]:
df.head()

,Resume_str,Category
0,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,HR
1,"HR SPECIALIST, US HR OPERATIONS ...",HR
2,HR DIRECTOR Summary Over 2...,HR
3,HR SPECIALIST Summary Dedica...,HR
4,HR MANAGER Skill Highlights ...,HR


In [26]:
# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Encode labels
df['Category'] = label_encoder.fit_transform(df['Category'])


In [28]:
# Convert DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Split the dataset into train and test sets
dataset = dataset.train_test_split(test_size=0.2)

        Preprocessing and Tokenization

In [29]:
# Load DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [30]:
# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['Resume_str'], padding="max_length", truncation=True)


In [31]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 497/497 [00:03<00:00, 133.51 examples/s]


In [32]:
# Rename columns for the model
tokenized_datasets = tokenized_datasets.rename_column('Category', 'labels')

        FineTuning BERT MODEL

In [33]:
# Define the model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(df['Category'].unique()))


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # Smaller batch size
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,
)


In [35]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
)

# Train the model
trainer.train()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
  1%|▏         | 10/747 [00:26<34:51,  2.84s/it]

{'loss': 3.1809, 'grad_norm': 2.9100518226623535, 'learning_rate': 1.9732262382864794e-05, 'epoch': 0.04}


  3%|▎         | 20/747 [00:57<36:54,  3.05s/it]

{'loss': 3.1491, 'grad_norm': 2.923863649368286, 'learning_rate': 1.9464524765729587e-05, 'epoch': 0.08}


  4%|▍         | 30/747 [01:28<25:40,  2.15s/it]

...

                                                 
100%|██████████| 747/747 [37:52<00:00,  2.18s/it]

{'eval_loss': 0.8317880630493164, 'eval_runtime': 28.9059, 'eval_samples_per_second': 17.194, 'eval_steps_per_second': 2.179, 'epoch': 3.0}


100%|██████████| 747/747 [37:54<00:00,  3.05s/it]

{'train_runtime': 2275.0019, 'train_samples_per_second': 2.62, 'train_steps_per_second': 0.328, 'train_loss': 1.5812881768467915, 'epoch': 3.0}


TrainOutput(global_step=747, training_loss=1.5812881768467915, metrics={'train_runtime': 2275.0019, 'train_samples_per_second': 2.62, 'train_steps_per_second': 0.328, 'train_loss': 1.5812881768467915, 'epoch': 3.0})

In [46]:
# Save the model
trainer.save_model('./resume_category_model')

In [41]:
from datasets import load_metric

# Load accuracy metric
metric = load_metric("accuracy")

# Define the compute_metrics function
def compute_metrics(p):
    predictions = p.predictions.argmax(axis=-1)  # Get predicted class indices
    return metric.compute(predictions=predictions, references=p.label_ids)

# Initialize Trainer with the compute_metrics function
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics
)

# Evaluate the model
eval_results = trainer.evaluate()

# Print evaluation results
print(f"Evaluation results: {eval_results}")

# Access the accuracy
accuracy = eval_results['eval_accuracy']
print(f"Accuracy: {accuracy:.4f}")


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
100%|██████████| 63/63 [00:32<00:00,  1.96it/s]

Evaluation results: {'eval_loss': 0.8317880630493164, 'eval_accuracy': 0.8591549295774648, 'eval_runtime': 40.7013, 'eval_samples_per_second': 12.211, 'eval_steps_per_second': 1.548}
Accuracy: 0.8592


        Testing Example

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load the fine-tuned model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained('./resume_category_model')
tokenizer = AutoTokenizer.from_pretrained('./resume_category_model')

# Define the category labels (update these based on your actual labels)
categories = [
    "Accountant", "Advocate", "Agriculture", "Apparell", "Arts", 
    "Automobile", "Aviation", "Banking", "BPO", "Business Development",
    "Chef", "Construction", "Consultant", "Designer", "Digital Media",
    "Engineering", "Finance", "Fitness", "Healthcare", "HR",
    "Information Technology", "Public Relations", "Sales", "Teacher"
]

def preprocess_text(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    return inputs

def predict(text):
    inputs = preprocess_text(text)

    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    return predictions.item()

def get_category_name(label_id):
    return categories[label_id]

# Example resume text
example_text = "Auditing and getting finances "
predicted_label_id = predict(example_text)
predicted_category = get_category_name(predicted_label_id)
print(f"The predicted category is: {predicted_category}")


The predicted category is: Banking
